In [81]:
import csv
import time
import requests
import pandas as pd

In [82]:
game_ids = list()
with open("Challenger_Ranked_Games_9-10k.csv") as csv_file:
    spam = csv.reader(csv_file, delimiter=",")
    for row in spam:
        game_ids.append(row[0])

num_of_games = len(game_ids)
print(f"There are {num_of_games} game ids.")


There are 2000 game ids.


In [83]:
qps = 100/120 # 100 query per 120 seconds
error_count = 0
corrupted_game_ids = list()

df = pd.DataFrame(columns=["game_id", "data"])

In [84]:
count = 0

for gid in game_ids[1:]: # first row is ignored since it is header
    if count % (num_of_games // 100) == 0:
        print(f"Progress: {round(count/num_of_games,3) * 100} %")
    count += 1
    # create the query
    q = f"https://kr.api.riotgames.com/lol/match/v4/matches/{gid}?api_key=RGAPI-05bd6712-e319-4be2-9f25-5529b995b4f5"
    
    # send get request
    r = requests.get(q)
    time.sleep(qps)
    
    # if error occurs
    while r.status_code != 200:
        print(f"Error in gid {gid}. {r.status_code}, {q}")
        time.sleep(qps)
        r = requests.get(q)
        error_count += 1
        if error_count == 5:
            corrupted_game_ids.append(gid)
            break
    error_count = 0
    
    # add game id and game_data to DataFrame
    df = df.append({"game_id": gid, "data": r.text}, ignore_index=True)

Progress: 0.0 %
Error in gid 4224499502. 504, https://kr.api.riotgames.com/lol/match/v4/matches/4224499502?api_key=RGAPI-05bd6712-e319-4be2-9f25-5529b995b4f5
Progress: 1.0 %
Progress: 2.0 %
Progress: 3.0 %
Progress: 4.0 %
Progress: 5.0 %
Progress: 6.0 %
Progress: 7.000000000000001 %
Progress: 8.0 %
Progress: 9.0 %
Progress: 10.0 %
Progress: 11.0 %
Progress: 12.0 %
Progress: 13.0 %
Progress: 14.000000000000002 %
Progress: 15.0 %
Progress: 16.0 %
Error in gid 4234600299. 504, https://kr.api.riotgames.com/lol/match/v4/matches/4234600299?api_key=RGAPI-05bd6712-e319-4be2-9f25-5529b995b4f5
Progress: 17.0 %
Progress: 18.0 %
Progress: 19.0 %
Progress: 20.0 %
Progress: 21.0 %
Progress: 22.0 %
Progress: 23.0 %
Progress: 24.0 %
Progress: 25.0 %
Progress: 26.0 %
Progress: 27.0 %
Progress: 28.000000000000004 %
Progress: 28.999999999999996 %
Progress: 30.0 %
Progress: 31.0 %
Error in gid 4231391845. 504, https://kr.api.riotgames.com/lol/match/v4/matches/4231391845?api_key=RGAPI-05bd6712-e319-4be2-9f

In [85]:
# save DataFrame as .csv file
df.to_csv("all_games9-10.csv")

In [86]:
print("These ids could not retrieved: ", corrupted_game_ids)

These ids could not retrieved:  []


In [68]:
df.size

1998

In [48]:
u = pd.DataFrame(columns=["game_id", "data"])

u["game_id"] = df["game_id"].unique()
u["data"] = df["data"].unique()[:-1]

u.to_csv("challenger_2k.csv")

ValueError: Length of values (998) does not match length of index (999)

In [45]:
u.shape

(9879, 2)